In [0]:
file_path = "dbfs:/user/hive/warehouse/group2_data.db/data_source"

df_combined = spark.read.format("delta").option("header", "true").load(file_path)

In [0]:
# display(df_combined.limit(5))

In [0]:
# from pyspark.sql.functions import col, count, when

# # Calculate the total number of rows in the dataframe
# total_rows = df_combined.count()

# # Calculate the number of empty rows for each column
# empty_row_counts = df_combined.select([(count(when(col(c).isNull(), c)).alias(c)) for c in df_combined.columns])

# # Calculate the percentage of empty rows for each column
# empty_row_percentages = empty_row_counts.select([(col(c) / total_rows * 100).alias(c) for c in empty_row_counts.columns])

# # Display the percentage of empty rows for each column
# display(empty_row_percentages)

In [0]:
selected_columns = ["enqueued_time_utc", "connection_device_id", "e_prog_runn", "e_prog_wait", "ch_1_prog_pfad_name", "ch_1_line_content", "ch_1_tool_ident","vorschuboverride", "eilgangoverride", "spindeloverride", "ch_1_act_speed_s_1", "nc_start_counter", "act_prog_net_time", "old_prog_net_time", "bag_1_op_mode"]



In [0]:
df = df_combined.select(selected_columns)

In [0]:
# display(df.limit(20))

In [0]:
# from pyspark.sql.functions import col, count, when

# # Calculate the total number of rows in the dataframe
# total_rows = df.count()

# # Calculate the number of empty rows for each column
# empty_row_counts = df.select([(count(when(col(c).isNull(), c)).alias(c)) for c in df.columns])

# # Calculate the percentage of empty rows for each column
# empty_row_percentages = empty_row_counts.select([(col(c) / total_rows * 100).alias(c) for c in empty_row_counts.columns])

# # Display the percentage of empty rows for each column
# display(empty_row_percentages)

In [0]:
# from pyspark.sql.functions import month, date_format, col

# # Extract the month from the date column
# months_df = df.select(date_format(col('enqueued_time_utc'), 'MMMM').alias('month'))

# # Get distinct months
# distinct_months = months_df.distinct()

# # Display the distinct months
# display(distinct_months)

In [0]:
# from pyspark.sql.functions import month

# # Filter rows for January
# df_january = df.filter(month(col('enqueued_time_utc')) == 1)

# # Get columns with non-null values in January
# non_null_columns = [c for c in df.columns if df_january.filter(col(c).isNotNull()).count() > 0]

# # Display the columns with data in January
# display(spark.createDataFrame([(c,) for c in non_null_columns], ["Column"]))

In [0]:
from pyspark.sql.functions import col
df_filtered = df.filter(
    col("enqueued_time_utc").isNotNull() &
    col("e_prog_runn").isNotNull() &
    col("e_prog_wait").isNotNull() &
    col("ch_1_prog_pfad_name").isNotNull() &
    col("ch_1_line_content").isNotNull() &
    col("ch_1_tool_ident").isNotNull() &
    col("vorschuboverride").isNotNull() &
    col("eilgangoverride").isNotNull() &
    col("spindeloverride").isNotNull() &
    col("ch_1_act_speed_s_1").isNotNull() &
    col("nc_start_counter").isNotNull() &
    col("act_prog_net_time").isNotNull() &
    col("old_prog_net_time").isNotNull()
)

In [0]:
# from pyspark.sql.functions import col, count, when

# # Calculate the total number of rows in the dataframe
# total_rows = df_filtered.count()

# # Calculate the number of empty rows for each column
# empty_row_counts = df_filtered.select([(count(when(col(c).isNull(), c)).alias(c)) for c in df_filtered.columns])

# # Calculate the percentage of empty rows for each column
# empty_row_percentages = empty_row_counts.select([(col(c) / total_rows * 100).alias(c) for c in empty_row_counts.columns])

# # Display the percentage of empty rows for each column
# display(empty_row_percentages)

In [0]:

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

class MachineProgramState:
    AUTOMATIC_ACTIVE = 'AUTOMATIC_ACTIVE'
    SETUP = 'SETUP'
    MANUAL = 'MANUAL'
    AUTOMATIC_INACTIVE = 'AUTOMATIC_INACTIVE'

def get_machine_state(e_prog_runn, e_prog_wait, bag_1_op_mode):
    if e_prog_runn or e_prog_wait:
        return MachineProgramState.AUTOMATIC_ACTIVE
    if bag_1_op_mode == 0:
        return MachineProgramState.SETUP
    elif bag_1_op_mode == 1:
        return MachineProgramState.MANUAL
    else:
        return MachineProgramState.AUTOMATIC_INACTIVE

get_machine_state_udf = udf(get_machine_state, StringType())

df_filtered2 = df_filtered.withColumn(
    "Machine_state", 
    get_machine_state_udf("e_prog_runn", "e_prog_wait", "bag_1_op_mode")
)



In [0]:
# display(df_filtered2.limit(50))

In [0]:
# from pyspark.sql.functions import month, date_format, col

# # Extract the month from the date column
# months_df = df_filtered2.select(date_format(col('enqueued_time_utc'), 'MMMM').alias('month'))

# # Get distinct months
# distinct_months = months_df.distinct()

# # Display the distinct months
# display(distinct_months)

month
November
January
October
December
September
August
February
March
May
April


In [0]:

from pyspark.sql.functions import hour, minute, dayofweek

df_filtered2 = df_filtered2.withColumn('day_of_week', dayofweek('enqueued_time_utc'))
df_filtered2 = df_filtered2.withColumn('hour', hour('enqueued_time_utc'))
df_filtered2 = df_filtered2.withColumn('minute', minute('enqueued_time_utc'))


In [0]:

from pyspark.sql.functions import when

df_filtered2 = df_filtered2.withColumn(
    'Shift',
    when((df_filtered2['hour'] >= 6) & (df_filtered2['hour'] < 14), 'First_shift')
    .when((df_filtered2['hour'] >= 14) & (df_filtered2['hour'] < 22), 'Second_shift')
    .otherwise('Night_shift')
)

In [0]:
# display(df_filtered2.limit(10))

In [0]:
import numpy as np
import pandas as pd

In [0]:

from pyspark.sql.functions import col, sin, cos, lit

df_filtered2 = df_filtered2.withColumn(
    'hour_sin', sin(2 * np.pi * col('hour') / lit(24))
).withColumn(
    'hour_cos', cos(2 * np.pi * col('hour') / lit(24))
).withColumn(
    'day_of_week_sin', sin(2 * np.pi * col('day_of_week') / lit(7))
).withColumn(
    'day_of_week_cos', cos(2 * np.pi * col('day_of_week') / lit(7))
).withColumn(
    'minute_sin', sin(2 * np.pi * col('minute') / lit(60))
).withColumn(
    'minute_cos', cos(2 * np.pi * col('minute') / lit(60))
)

In [0]:
display(df_filtered2.limit(100))

enqueued_time_utc,connection_device_id,e_prog_runn,e_prog_wait,ch_1_prog_pfad_name,ch_1_line_content,ch_1_tool_ident,vorschuboverride,eilgangoverride,spindeloverride,ch_1_act_speed_s_1,nc_start_counter,act_prog_net_time,old_prog_net_time,bag_1_op_mode,Machine_state,day_of_week,hour,minute,Shift,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,minute_sin,minute_cos
2022-10-22T06:09:31.088Z,berlin-12445,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,T16_1730,0.0,0.0,100.0,0.0,290.0,0.0,0.0,1.0,MANUAL,7,6,9,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,0.8090169943749475,0.5877852522924731
2022-10-22T06:09:31.775Z,niederahr-28518601,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,T16_1730,0.0,0.0,100.0,0.0,290.0,0.0,0.0,1.0,MANUAL,7,6,9,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,0.8090169943749475,0.5877852522924731
2022-10-22T06:09:35.573Z,koszalin-00010,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,T16_1730,0.0,0.0,100.0,0.0,290.0,0.0,0.0,1.0,MANUAL,7,6,9,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,0.8090169943749475,0.5877852522924731
2022-10-22T06:15:42.243Z,oelde-35114,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,T16_1730,0.0,0.0,100.0,0.0,290.0,0.0,0.0,1.0,MANUAL,7,6,15,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,1.0,2.83276944882399E-16
2022-10-22T06:15:47.166Z,koszalin-00008,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,T16_1730,0.0,0.0,100.0,0.0,290.0,0.0,0.0,1.0,MANUAL,7,6,15,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,1.0,2.83276944882399E-16
2022-10-22T06:16:21.223Z,oelde-35114,false,false,/_N_WKS_DIR/_N_H_BC0214_GEA_GEHAEUSE_WPD/_N_1000_MPF,,,100.0,100.0,100.0,0.0,208.0,0.0,1113.3120000000001,0.0,SETUP,7,6,16,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,0.9945218953682734,-0.10452846326765333
2022-10-22T06:16:26.068Z,koszalin-00008,false,false,/_N_WKS_DIR/_N_H_BC0214_GEA_GEHAEUSE_WPD/_N_1000_MPF,,,100.0,100.0,100.0,0.0,208.0,0.0,1113.3120000000001,0.0,SETUP,7,6,16,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,0.9945218953682734,-0.10452846326765333
2022-10-22T06:16:30.225Z,koszalin-00001,false,false,/_N_WKS_DIR/_N_H_BC0214_GEA_GEHAEUSE_WPD/_N_1000_MPF,,,100.0,100.0,100.0,0.0,208.0,0.0,1113.3120000000001,0.0,SETUP,7,6,16,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,0.9945218953682734,-0.10452846326765333
2022-10-22T06:16:30.397Z,berlin-12445,false,false,/_N_WKS_DIR/_N_H_BC0214_GEA_GEHAEUSE_WPD/_N_1000_MPF,,,100.0,100.0,100.0,0.0,208.0,0.0,1113.3120000000001,0.0,SETUP,7,6,16,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,0.9945218953682734,-0.10452846326765333
2022-10-22T06:16:32.412Z,niederahr-28518601,false,false,/_N_WKS_DIR/_N_H_BC0214_GEA_GEHAEUSE_WPD/_N_1000_MPF,,,100.0,100.0,100.0,0.0,208.0,0.0,1113.3120000000001,0.0,SETUP,7,6,16,First_shift,1.0,6.123233995736766E-17,-2.4492935982947064E-16,1.0,0.9945218953682734,-0.10452846326765333


In [0]:
df_sorted = df_filtered2.orderBy(col('enqueued_time_utc'))
display(df_sorted.limit(100))

enqueued_time_utc,connection_device_id,e_prog_runn,e_prog_wait,ch_1_prog_pfad_name,ch_1_line_content,ch_1_tool_ident,vorschuboverride,eilgangoverride,spindeloverride,ch_1_act_speed_s_1,nc_start_counter,act_prog_net_time,old_prog_net_time,bag_1_op_mode,Machine_state,day_of_week,hour,minute,Shift,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,minute_sin,minute_cos
2022-01-01T00:16:42.116Z,oelde-11319,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,,0.0,0.0,100.0,0.0,101.0,0.0,3669.291,0.0,SETUP,7,0,16,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9945218953682734,-0.10452846326765333
2022-01-01T00:16:49.569Z,oelde-36103,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,,0.0,0.0,100.0,0.0,101.0,0.0,3669.291,0.0,SETUP,7,0,16,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9945218953682734,-0.10452846326765333
2022-01-01T00:17:03.398Z,oelde-11319,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,,0.0,0.0,100.0,0.0,101.0,0.0,3669.291,0.0,SETUP,7,0,17,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9781476007338057,-0.20791169081775934
2022-01-01T00:17:09.851Z,oelde-36103,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,,0.0,0.0,100.0,0.0,101.0,0.0,3669.291,0.0,SETUP,7,0,17,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9781476007338057,-0.20791169081775934
2022-01-01T00:17:23.033Z,oelde-11319,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,,0.0,0.0,100.0,0.0,101.0,0.0,3669.291,0.0,SETUP,7,0,17,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9781476007338057,-0.20791169081775934
2022-01-01T00:17:42.315Z,oelde-11319,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,,0.0,0.0,100.0,0.0,101.0,0.0,3669.291,0.0,SETUP,7,0,17,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9781476007338057,-0.20791169081775934
2022-01-01T00:18:02.253Z,oelde-11319,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,,0.0,0.0,100.0,0.0,101.0,0.0,3669.291,0.0,SETUP,7,0,18,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9510565162951536,-0.30901699437494734
2022-01-01T00:18:22.21Z,oelde-11319,false,false,/_N_SYF_DIR/_N_OSTORE1_SYF,,,0.0,0.0,100.0,0.0,101.0,0.0,3669.291,0.0,SETUP,7,0,18,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9510565162951536,-0.30901699437494734
2022-01-01T00:18:42.882Z,oelde-11319,false,false,/_N_WKS_DIR/_N_GEH_MCI_WPD/_N_RSG_HM2_UNI_M1_MPF,,,0.0,0.0,100.0,0.0,1265.0,0.0,23872.035,2.0,AUTOMATIC_INACTIVE,7,0,18,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9510565162951536,-0.30901699437494734
2022-01-01T00:18:48.742Z,oelde-36103,false,false,/_N_WKS_DIR/_N_GEH_MCI_WPD/_N_RSG_HM2_UNI_M1_MPF,,,0.0,0.0,100.0,0.0,1265.0,0.0,23872.035,2.0,AUTOMATIC_INACTIVE,7,0,18,Night_shift,0.0,1.0,-2.4492935982947064E-16,1.0,0.9510565162951536,-0.30901699437494734


In [0]:
# df_sorted.write.saveAsTable("group2.machine_state_table_combined")